# **Required Libraries**

--------------------------------------------------------------------------------

In [1]:
!pip3 install python-Wappalyzer
!pip3 install nslookup
!pip3 install python-nmap

In [2]:
import subprocess
import os
import requests
import json
import re
import nmap
import shlex
import socket
import ssl
import ast


from nslookup import Nslookup
from Wappalyzer import Wappalyzer, WebPage

domain = ""
active_subdomains = []

#LLM
from openai import OpenAI 
from sk import my_sk

#Required Libraries for RAG
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate


/home/izack05/Desktop/testing/.env/lib/python3.13/site-packages/Wappalyzer/Wappalyzer.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
def create_scan_result_folder(folder_name):
    path = r"/home/izack05/Desktop/testing"
    os.chdir(path)
    os.makedirs(folder_name, exist_ok = True)

In [4]:
create_scan_result_folder("results")
create_scan_result_folder("wapiti_result")
create_scan_result_folder("action_plan_results")
create_scan_result_folder("html_results")

# **Reconnaisance Module**

--------------------------------------------------------------------------------

In [3]:
def certificate_data_lookup(domain):
    print("[#] Starting certificate data lookup:")
    domain_pattern = re.compile(r'\b(?:[a-zA-Z0-9-]*\.)+[a-zA-Z]{2,}\b') 
    cert_subdomains = []
    try:
        query = f"https://crt.sh/?q={domain}&output=json"
        response = requests.get(query)

        #print(response.status_code)
        if response.status_code == 200:
            #print(response.text)
            response_dict = json.loads(response.text)
            for i in response_dict:    
                value = domain_pattern.findall(i["name_value"])
                for i in value:    #because value = ["something.com","anything.com"]  
                    if i not in cert_subdomains:
                        cert_subdomains.append(i)
        else:
            print("Error in https query for domain data lookup")
        print("")
        print(f"Number of subdomains found from public Lookup is : {len(cert_subdomains)}")
        # for i in cert_subdomains: 
        #     print(f"-{i}")

        for index, i in enumerate(cert_subdomains):
            if i.startswith("www."):
                cert_subdomains[index] = i.replace("www.", "")       

        return cert_subdomains

    except Exception as e:
        print(f"Public data lookup for {domain} failed: {e}") 

In [4]:
def toolEnum(domain):
    #domain = "chaldal.com"
    print("[#] Starting subdomain enumeration using tool:")
    domain_pattern = re.compile(r'\b(?:[a-zA-Z0-9-]*\.)+[a-zA-Z]{2,}\b')

    try:
        subdomains = subprocess.check_output(f"subfinder -d {domain} -all -t 1 -rl  1", text=True, shell=True)
        tool_subdomains = domain_pattern.findall(subdomains)

        for index, i in enumerate(tool_subdomains):
            if i.startswith("www."):
                tool_subdomains[index] = i.replace("www.", "")

        print(f"Number of subdomains found using tool is : {len(tool_subdomains)}\n")
        # for i in tool_subdomains: 
        #     print(f"-{i}")
        
        return tool_subdomains

    except subprocess.CalledProcessError as e:
        print(f"Subdomain emuneration via tool failed with return code {e.returncode}")

In [5]:
def domainActiveCheck(domains):
    active_domains = []
    
    for domain in domains:
        try:
            query1 = f"http://{domain}"

            response1 = requests.get(query1) #, verify=False)
            #print(f"{domain} = {response1.status_code}")
            if response1.status_code < 400:
                active_domains.append(domain)
                
            query2 = f"https://{domain}"
            response2 = requests.get(query2)
            #print(f"{domain} = {response2.status_code}")
            if response2.status_code < 400 and domain not in active_domains:
                active_domains.append(domain)
                
        except requests.RequestException:
            continue
    return active_domains

In [6]:

def get_ip(subdomains):
    global recon_info
    print("[#] Fetcing IP addresses: ")
    dns_query = Nslookup()
    ips = []
    
    for domain in subdomains:
        try:
            ips_record = dns_query.dns_lookup(domain)
            print(f"{domain} = {ips_record.answer}")
            recon_info += f"{domain} = {ips_record.answer}\n"
            #print(ips_record.answer)
            for ip in ips_record.answer:       #because ips_record = [11.222.44, 108.292.96] multiple ips
                if ip not in ips:
                    ips.append(ip)
        except  Exception as e:
            print("dns resolving failed :(( ") 
            
    return ips

In [7]:
def find_open_ports(active_ips):
    print("[#] Searching for open ports: ")
    nm = nmap.PortScanner()
    open_ports_dict = {}

    for ip in active_ips:
        try:
            nm.scan(ip, '1-1024')
            open_ports = []
            if ip in nm.all_hosts() and nm[ip].state() == 'up':
                for proto in nm[ip].all_protocols():
                    ports = nm[ip][proto].keys()
                    open_ports.extend(sorted(ports))

            open_ports_dict[ip] = open_ports

        except Exception as e:
            print(f"Error scanning {ip}: {e}")
            open_ports_dict[ip] = []
    
    return open_ports_dict

In [8]:
def techstack(domain):
    try:
        wappalyzer = Wappalyzer.latest()
        query = f"http://{domain}"
        webpage = WebPage.new_from_url(query)
        #wappalyzer.analyze(webpage)
        wappalyzer.analyze_with_categories(webpage)
        print()

        return wappalyzer.analyze_with_versions_and_categories(webpage)
    except Exception as e:
        print(f"Error in fetching tech stack of {domain}: {e}")

In [9]:
def FireWall_Check(domain):
    # try: 
    #     detection_result = subprocess.check_output(f"wafw00f http://{domain} -v", text=True, shell=True)
    #     return detection_result
    try: 
        detection_result = subprocess.check_output(f"wafw00f http://{domain} -v", text=True, shell=True)
        start_index = detection_result.find("[*] Checking")
        if start_index != -1:
            result = detection_result[start_index:]
            return result
        
    except subprocess.CalledProcessError as e:
        print(f"Command failed with return code {e.returncode}")

In [10]:
def get_parameters(domain):
    try:
        # Tool name
        tool_name = "paramspider"

        # Run the command   [this stores the result in a txt file in tool directory]
        result = subprocess.run(f"{tool_name} -d www.{domain}", capture_output=True, text=True, shell=True)

        # Output the result
        print(f"[#] Parameter Fetching for {domain} completed\nCheck tool directory for results.\n")
        # results being stored at path --> /home/izack05/results/{domain}.txt
    except Exception as e:
        print(f"Parameter fetching failed: {e}")

In [11]:
def check_ssl_support(domain, timeout=5):
    port = 443
    context = ssl.create_default_context()

    try:
        with socket.create_connection((domain, port), timeout=timeout) as sock:
            with context.wrap_socket(sock, server_hostname=domain) as ssock:
                status = f"[#] {domain} supports SSL/TLS ({ssock.version()})"
                #print(status)
                return [True, status]
    except (socket.timeout, ConnectionRefusedError, ssl.SSLError, ssl.CertificateError, OSError) as e:
        status = f"[X] {domain} does NOT support SSL/TLS ({e.__class__.__name__})" 
        #print(status)
        return [False, status]

# **Reconnaissance Driver Code**
--------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
#-----DRIVER------
global domain, active_subdomains
recon_info = ""
subdomains = []

domain = "testphp.vulnweb.com"
#domain = "127.0.0.1/DVWA/"
recon_info += f"[#] Main domain is '{domain}' \n\n"

print("[#] Reconnaissance Module \n")
print(f"[#] Starting reconnaissance for the domain = {domain}\n")

certificate_data_lookup_subdomains = certificate_data_lookup(domain)
tool_subdomains = toolEnum(domain)

subdomains.append(domain)
for subDT in tool_subdomains:
    if subDT not in subdomains:
        subdomains.append(subDT)

for subDP in certificate_data_lookup_subdomains:
    if subDP not in subdomains:
        subdomains.append(subDP)

print(f"Total no. of unique subdomains found:{len(subdomains)}")
print(f"subdomain list:{subdomains}")

active_subdomains = domainActiveCheck(subdomains)
print(f"[#] Total no. of active subdomains found is : {len(active_subdomains)}")
recon_info += f"\nTotal no. of active subdomains found is : {len(active_subdomains)}\n"
recon_info += f"[#] List of the subdomains: {active_subdomains}\n"

for i in active_subdomains: 
    print(f"-{i}")

print("\n")

recon_info += f"\n[#] Active Subdomains and their IP addresses are:\n"
active_ips = get_ip(active_subdomains)   # it is getting appened at the get_ip function

print("[#]Active and unique IP addresses are:")
for i in active_ips:
    print(f"- {i}")

print("")
open_ports_dict = find_open_ports(active_ips)
print(f"[#] Open ports associated with each unique IP address:\n{open_ports_dict}")
recon_info += f"\n\nOpen ports associated with each unique IP address:\n{open_ports_dict}"

tech_stack = techstack(domain)
recon_info += f"\n\nTechnology stack of the main domain: {domain}:\n"
print(f"[#] Technology stack of the main domain:{domain} are:")
if tech_stack != None:
    for key, value in tech_stack.items():
        print(f"{key} : {value}")
        recon_info += f"{key} : {value}\n"

firewallResult = FireWall_Check(domain)
print("\nFirewall detection result:")
print(firewallResult)
recon_info += f"\nFirewall detection result: \n{firewallResult}"

ssl_status = check_ssl_support(domain)
print("\nSSL/TLS Certification result:")
print(ssl_status)
recon_info += f"\nSSL/TLS Certification result: \n{ssl_status}\n\n"
get_parameters(domain)


[#] Reconnaissance Module 

[#] Starting reconnaissance for the domain = testphp.vulnweb.com

[#] Starting certificate data lookup:
Error in https query for domain data lookup

Number of subdomains found from public Lookup is : 0
[#] Starting subdomain enumeration using tool:



               __    _____           __         
   _______  __/ /_  / __(_)___  ____/ /__  _____
  / ___/ / / / __ \/ /_/ / __ \/ __  / _ \/ ___/
 (__  ) /_/ / /_/ / __/ / / / / /_/ /  __/ /    
/____/\__,_/_.___/_/ /_/_/ /_/\__,_/\___/_/

		projectdiscovery.io

[INF] Current subfinder version v2.6.0 (outdated)
[INF] Loading provider config from /home/izack05/.config/subfinder/provider-config.yaml
[INF] Enumerating subdomains for testphp.vulnweb.com
[INF] Found 2 subdomains for testphp.vulnweb.com in 14 seconds 481 milliseconds


Number of subdomains found using tool is : 2

Total no. of unique subdomains found:2
subdomain list:['testphp.vulnweb.com', 'sieb-web1.testphp.vulnweb.com']
[#] Total no. of active subdomains found is : 1
-testphp.vulnweb.com


[#] Fetcing IP addresses: 
testphp.vulnweb.com = ['44.228.249.3']
[#]Active and unique IP addresses are:
- 44.228.249.3

[#] Searching for open ports: 
[#] Open ports associated with each unique IP address:
{'44.228.249.3': [53, 80]}


/home/izack05/Desktop/testing/.env/lib/python3.13/site-packages/Wappalyzer/Wappalyzer.py:226: UserWarning: Caught 'unbalanced parenthesis at position 119' compiling regex: ['(?:<div class="sf-toolbar[^>]+?>[^]+<span class="sf-toolbar-value">([\\d.])+|<div id="sfwdt[^"]+" class="[^"]*sf-toolbar)', 'version:\\1']
  warnings.warn(



[#] Technology stack of the main domain:testphp.vulnweb.com are:
Ubuntu : {'versions': [], 'categories': ['Operating systems']}
PHP : {'versions': ['5.6.40'], 'categories': ['Programming languages']}
DreamWeaver : {'versions': [], 'categories': ['Editors']}
Nginx : {'versions': ['1.19.0'], 'categories': ['Web servers', 'Reverse proxies']}

Firewall detection result:
[*] Checking http://testphp.vulnweb.com
[+] Generic Detection results:
[-] No WAF detected by the generic detection
[~] Number of requests: 7


SSL/TLS Certification result:
[False, '[X] testphp.vulnweb.com does NOT support SSL/TLS (TimeoutError)']
[#] Parameter Fetching for testphp.vulnweb.com completed
Check tool directory for results.



In [14]:
global recon_info, domain

print(recon_info)
path = r"/home/izack05/Desktop/testing/results"
os.chdir(path)
safe_domain = domain.replace("/", "_")
recon_info_file = f"{safe_domain}_recon_information.txt"
with open(recon_info_file, "w", encoding="utf-8") as wf:
    for char in recon_info:
        wf.write(char)

path = r"/home/izack05/Desktop/testing/results"
os.chdir(path)  

[#] Main domain is 'testphp.vulnweb.com' 


Total no. of active subdomains found is : 1
[#] List of the subdomains: ['testphp.vulnweb.com']

[#] Active Subdomains and their IP addresses are:
testphp.vulnweb.com = ['44.228.249.3']


Open ports associated with each unique IP address:
{'44.228.249.3': [53, 80]}

Technology stack of the main domain: testphp.vulnweb.com:
Ubuntu : {'versions': [], 'categories': ['Operating systems']}
PHP : {'versions': ['5.6.40'], 'categories': ['Programming languages']}
DreamWeaver : {'versions': [], 'categories': ['Editors']}
Nginx : {'versions': ['1.19.0'], 'categories': ['Web servers', 'Reverse proxies']}

Firewall detection result: 
[*] Checking http://testphp.vulnweb.com
[+] Generic Detection results:
[-] No WAF detected by the generic detection
[~] Number of requests: 7

SSL/TLS Certification result: 
[False, '[X] testphp.vulnweb.com does NOT support SSL/TLS (TimeoutError)']




# **Vulnerability Assessment Scan Module**

--------------------------------------------------------------------------------

In [57]:
def clean_ansi_escape_codes(text):
    # This regular expression matches ANSI escape sequences
    ansi_escape_pattern = r'\x1b\[[0-9;]*[mHfK]'
    
    # Substitute the ANSI escape sequences with an empty string
    cleaned_text = re.sub(ansi_escape_pattern, '', text)
    
    return cleaned_text


In [58]:
def directory_traversal_scan(domain):
    global vulnerability_assessment_scan_results, vulnerability_assessment_scan_dict
    
    print("[#]Starting path Directory traversal:\n")

    wordlist_path = "/usr/share/wordlists/dirb/common.txt"

    # file discovery/directory traversal
    browsers = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
    gobuster_result_path = "~/Desktop/testing/gobuster_result/gobuster_result.txt"

    try:
        ans = subprocess.check_output(f"gobuster dir -u {domain} -w {wordlist_path} -t 5 -a '{browsers}' -o {gobuster_result_path}", text=True, shell=True)
        #print(ans)
        with open(r"/home/izack05/Desktop/testing/gobuster_result/gobuster_result.txt", "r", encoding="utf-8") as rf:
            a = rf.read()
            #print(a)

        vulnerability_assessment_scan_results += "[#] Path Directory traversal scan results: \n"
        vulnerability_assessment_scan_results += a
        vulnerability_assessment_scan_results += f"[#] Path directory traversal scan finished.\n\n"

        vulnerability_assessment_scan_dict["Directory Path Traversal"] = a
        print(f"[#] Directory traversal scan done and results saved in: /home/izack05/Desktop/testing/gobuster_result/gobuster_result.txt")

    except subprocess.CalledProcessError as e:
        print(f"Directory traversal failed: {e.returncode}")
    

In [59]:
def xss_scan(domain):
    global vulnerability_assessment_scan_results, vulnerability_assessment_scan_dict
    
    print("[#]Starting XSS scan: \n")
    # Directory containing the tool
    tool_directory = r"/home/izack05/Desktop/pwnxss/PwnXSS"
    os.chdir(tool_directory)

    # Run the command
    try:
        ans = subprocess.check_output(f"python3 pwnxss.py -u {domain}/", text=True, shell=True, encoding='utf-8')
        #print(ans)
        text = clean_ansi_escape_codes(ans)
        
        pattern = r'^\[.*\]\s*\[CRITICAL\].*'
        
        # Find all lines that match the pattern
        critical_lines = re.findall(pattern, text, re.MULTILINE)
        # print(critical_lines)
        # for i in critical_lines:
        #     print(i)

        a = ""
        vulnerability_assessment_scan_results += "[#] XSS scan results:\n"

        for match in critical_lines:
            vulnerability_assessment_scan_results += f"{match}\n"
            a += f"{match}\n"

        vulnerability_assessment_scan_results += f"[#] XSS scan finished.\n\n"
        
        vulnerability_assessment_scan_dict["XSS"] = a

    except subprocess.CalledProcessError as e:
        print(f"XSS scan error: {e.returncode}")

    tool_directory = r"/home/izack05/Desktop/testing"
    os.chdir(tool_directory)
    print(f"[#] XSS scan finished.\n\n")

    

In [60]:
def nikto_scan(domain):
    global vulnerability_assessment_scan_results, vulnerability_assessment_scan_dict

    domain_clean = domain.replace("http://", "").replace("https://", "").strip("/")
    safe_domain = domain_clean.replace(".", "_")

    # output_folder = "/home/izack05/Desktop/testing/nikto"
    # os.makedirs(output_folder, exist_ok=True)

    #output_path = os.path.join(output_folder, f"nikto_result_{safe_domain}.txt")
    cmd = ["nikto", "-host", domain, "-ask","no"]

    print(f"[#] Running Nikto scan for: {domain_clean}")
    try:
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        stdout, stderr = process.communicate()

        # if process.returncode == 0:
        #     print("[#] Nikto scan completed successfully.")
        #     print(stdout)
        # else:
        #     print(f"[!] Nikto scan failed with return code {process.returncode}")
        #     print("[!] STDERR:")
        #     print(stderr)
        #     print("[!] STDOUT:")
        #     print(stdout)
        #print(stdout)
        vulnerability_assessment_scan_results += "[#] Nikto web server vulnerability scan results starts [might contain: dangerous file info, outdated http headers, server misconfiguration info and more]: \n"
        vulnerability_assessment_scan_results += stdout
        vulnerability_assessment_scan_results += f"[#] Nikto web server vulnerability scan finished.\n\n"

        vulnerability_assessment_scan_dict["Nikto scan (might contain: dangerous file info, outdated http headers, server misconfiguration info and more)"] = stdout
        print(f"[#] Nikto scan finished")
    except subprocess.CalledProcessError as e:
        print(f"Open redirection scan error: {e.returncode}")


In [61]:
def weak_cipher_scan(domain_sent):
    global vulnerability_assessment_scan_results, vulnerability_assessment_scan_dict
    domain_clean = domain_sent.replace("http://", "").replace("https://", "").strip("/")
    domain = domain_clean.replace(".", "_")
    check = check_ssl_support(domain)

    if check[0] == True:
        print(check[1])
        print("[#] Starting weak cipher scan:")

        # Run the command
        try:
            ans = subprocess.check_output(f"testssl {domain}/", text=True, shell=True, encoding='utf-8')
            text = clean_ansi_escape_codes(ans)
            #print(text)

            vulnerability_assessment_scan_results += "[#] weak cipher scan results:\n"
            vulnerability_assessment_scan_results += f"{check[1]}\n"
            vulnerability_assessment_scan_results += text
            vulnerability_assessment_scan_results += "[#] Weak cipher scan results end.\n"

            vulnerability_assessment_scan_dict["Weak Cipher"] = f"{check[1]}\n" +text

            # for match in critical_lines:
            #     vulnerability_assessment_scan_results += f"{match}\n"

            # vulnerability_assessment_scan_results += f"[#] weak cipher scan finished.\n\n"
        except subprocess.CalledProcessError as e:
            print(f"Weak cipher scan error: {e.returncode}")

        print(f"[#] Weak cipher scan finished.\n\n")
    else:
        print(f"{check[1]}. Therefore, weak cipher scan not required")
        vulnerability_assessment_scan_results += f"{check[1]}. Therefore, weak cipher scan not required.\n\n\n"
        vulnerability_assessment_scan_dict["Weak Cipher"] = f"{check[1]}. Therefore, weak cipher scan not required.\n\n"
        

**wapiti scan**
_____________________________

In [62]:
def parse_wapiti_report(text: str):
    """
    Find headings consisting of a line followed immediately by a dashed line (---)
    and capture the body until the next such heading or end of text.
    Returns a dict mapping header -> body (preserves formatting).
    """
    pattern = r'(?m)^(?P<header>[^\n\r]+)\n-+\n(?P<body>.*?)(?=\n^[^\n\r]+?\n-+\n|\Z)'
    matches = re.finditer(pattern, text, flags=re.S|re.M)
    result = {}
    for m in matches:
        header = m.group('header').strip()
        body = m.group('body').rstrip()
        result[header] = body
    return result

In [63]:
def wapiti_scan(domain):
    global parsed, vulnerability_assessment_scan_results, vulnerability_assessment_scan_dict

    print("[#] Starting Wapiti scan...\n")

    path = r"/home/izack05/Desktop/testing/wapiti_result"
    os.chdir(path)
    
    # always have to use the "backup" module first because it helps in data extraction in later cells
    try:
        run_command = subprocess.check_output(
            f"wapiti -u '{domain}/' -m backup,blindsql,brute_login_form,cookieflags,crlf,csp,csrf,exec,redirect,sql,ssrf,xxe,xss -f txt -o wapiti_output.txt",
            text=True,
            shell=True
        )
        
        with open("wapiti_output.txt") as f:
            report = f.read()

        end = report.find("Summary of anomalies")  #-->starting and ending line of the report #1975 is the first num of chars we ignore
        parsed = parse_wapiti_report(report[1975:end])    #-->sliced input for better data extraction ## a dictionary is sent back here

        #print(parsed)
        if vulnerability_assessment_scan_dict["XSS"]:
            parsed.pop("Cross Site Scripting", None)
        for k,v in parsed.items():
            # print(f"{k}:-->\n")
            # print(v)

            vulnerability_assessment_scan_results += f"\n\n[#] {k} scan results start:\n"
            vulnerability_assessment_scan_results += v
            vulnerability_assessment_scan_results += f"\n[#] {k} scan results end.\n\n"
            vulnerability_assessment_scan_dict.update(parsed)
        

    except subprocess.CalledProcessError as e:
        print(f"[!] Wapiti scan failed with return code {e.returncode}")
    
    path = r"/home/izack05/Desktop/testing/"
    os.chdir(path)
    print("[#] Starting Wapiti scan finished\n\n")
    

# **Driver Code for vulnerability assessment scans**
------------------------------------------------------------------------------------------------------------------------------------

In [64]:
#driver_vulnerability_assessment_scans
vulnerability_assessment_scan_results = ""
main_vulnerability_assessment_scan_dict = {}

folder_name = "vulnerability_assessment_result"
create_scan_result_folder(folder_name)

global domain, active_subdomains

print(active_subdomains)
print(f"[#] Vulnerability Assessment Scan for all active subdomains of domain {domain}")
vulnerability_assessment_scan_results += f"[#] Vulnerability Assessment Scan for all active subdomains of domain {domain}"

for sub_dom in active_subdomains:
    vulnerability_assessment_scan_dict = {}

    print(f"[#] Starting vulnerablilty assessment scan for {sub_dom}\n")
    vulnerability_assessment_scan_results += f"\n\n[-] Starting vulnerablilty assessment scan for subdomain = {sub_dom}\n"

    if not sub_dom.startswith(("http://", "https://")):
        sub_dom = "http://" + sub_dom
        
    if sub_dom[len(sub_dom)-1:len(sub_dom)] == "/":
        sub_dom = sub_dom[:len(sub_dom)-1]

    directory_traversal_scan(sub_dom)

    xss_scan(sub_dom)

    #run_sqlmap(sub_dom)

    nikto_scan(sub_dom)

    weak_cipher_scan(sub_dom)

    wapiti_scan(sub_dom)

    vulnerability_assessment_scan_results += f"[#] Vulnerablilty assessment scan finished for subdomain = {sub_dom}\n"
    print(f"[#] Vulnerablilty assessment scan finished for subdomain = {sub_dom}\n")
    main_vulnerability_assessment_scan_dict[sub_dom] = vulnerability_assessment_scan_dict





['testphp.vulnweb.com']
[#] Vulnerability Assessment Scan for all active subdomains of domain testphp.vulnweb.com
[#] Starting vulnerablilty assessment scan for testphp.vulnweb.com

[#]Starting path Directory traversal:



Progress: 4614 / 4615 (99.98%)

[#] Directory traversal scan done and results saved in: /home/izack05/Desktop/testing/gobuster_result/gobuster_result.txt
[#]Starting XSS scan: 

[#] XSS scan finished.


[#] Running Nikto scan for: testphp.vulnweb.com
[#] Nikto scan finished
[X] testphp_vulnweb_com does NOT support SSL/TLS (gaierror). Therefore, weak cipher scan not required
[#] Starting Wapiti scan...

[#] Starting Wapiti scan finished


[#] Vulnerablilty assessment scan finished for subdomain = http://testphp.vulnweb.com



**Extracting and removing duplicates from scan results and parsing (Clean Output)**
_______

In [65]:
# Finding vulnerablity count
def extract_blocks_general(text):
    """
    Split the text on lines with 3+ '*' (separator) and return all non-empty blocks,
    deduplicated while preserving order.
    """
    parts = re.split(r'^\s*(?:\*\s*){3,}\s*$',text, flags=re.MULTILINE)
    results = []
    for p in parts:
        chunk = p.strip()
        if chunk and chunk not in results:
            results.append(chunk)
    return results

def extract_lines(text):
    seen = set()
    out = []
    for raw in text.splitlines():
        line = raw.strip()
        if not line:
            continue
        if line not in seen:
            out.append(line)
            seen.add(line)
    return out

def unique_xss(text):
    pattern = re.compile(r'^\[.*?\]\s*(\[\s*CRITICAL\s*\].*)', re.MULTILINE | re.IGNORECASE)
    matches = pattern.findall(text)
    fin = []
    for m in matches:
        if m not in fin:
            fin.append(m)
            #print(m)
    return fin

def extract_nikto_count(text):
    m = re.search(r'\d+\s+error\(s\)\s+and\s+(\d+)\s+item\(s\)\s+reported', text, re.IGNORECASE)
    items_count = int(m.group(1)) if m else None
    #print(items_count)
    return items_count


In [66]:
global domain, vulnerability_assessment_scan_results, main_vulnerability_assessment_scan_dict

curated_vulnerabliity_scan_dict = {}
f = ""    #curated vulnerability scan result variable

not_done_by_wapiti = ["Directory Path Traversal", "XSS", "Nikto scan (might contain: dangerous file info, outdated http headers, server misconfiguration info and more)", "Weak Cipher"]

f += f"[#] Vulnerability Assessment Scan for all active subdomains of domain {domain}"

for key, value in main_vulnerability_assessment_scan_dict.items():
    scan_dict = value

    f += f"\n\n[-] Vulnerablilty assessment scan start for subdomain = {key}\n"

    for k, v in scan_dict.items():
        if k not in not_done_by_wapiti:
            sample = v        
            blocks = extract_blocks_general(sample)
            curated_vulnerabliity_scan_dict[k] = blocks
            f += "\n\n-------------------------------------------\n"
            f += f"\n[#] {k} scan results start:\n"
            f += f" -> found {len(blocks)} unique endpoints for {k}\n\n"
            for i,b in enumerate(blocks,1):
                f +=f"--- Endpoint {i} ---\n{b}\n\n"
            f += f"[#] {k} scan results end.\n\n"
            #print("\n\n-------------------------------------------\n")
            #print(f"Found {len(blocks)} unique blocks for {k}\n")
            # for i,b in enumerate(blocks,1):
            #     print(f"--- Block {i} ---\n{b}\n")

        else:
            if k == "Directory Path Traversal":
                lines = extract_lines(v)
                curated_vulnerabliity_scan_dict[k] = lines
                f += "\n[#] Path Directory traversal scan results:\n\n"
                for i in lines:
                    f += f"{i}\n"
                f += "\n[#] Path directory traversal scan finished.\n\n"
                # print(f"Found {len(lines)} lines")
                # for i,l in enumerate(lines,1):
                #     print(i, l)

            elif k == "XSS":
                xss_points = unique_xss(v)
                curated_vulnerabliity_scan_dict[k] = xss_points
                f += "[#] XSS scan results:\n"
                for j in xss_points:
                    f += f'{j}\n'
                f += "[#] XSS scan finished.\n\n"

            elif k == "Nikto scan (might contain: dangerous file info, outdated http headers, server misconfiguration info and more)":
                curated_vulnerabliity_scan_dict[k] = v
                f += "[#] Nikto web server vulnerability scan results starts [might contain: dangerous file info, outdated http headers, server misconfiguration info and more]: \n"
                num = extract_nikto_count(v)
                f += f"No. of Vulnerabilities found: {num}\n"
                f += v
                f += "[#] Nikto web server vulnerability scan finished.\n\n"
            elif k == "Weak Cipher":
                curated_vulnerabliity_scan_dict[k] = v
                f += "[#] Weak Cipher scan results: \n"
                f += v
                f += "[#] Weak Cipher scan finished.\n\n"
            else:
                print(f"not found output for {k}")
                
    f += f"[#] Vulnerablilty assessment scan finished for subdomain = {key}\n"

vulnerability_assessment_scan_results = f

**Writing Scan results in a file**
________________

In [67]:
global vulnerability_assessment_scan_results, main_vulnerability_assessment_scan_dict, domain

path = r"/home/izack05/Desktop/testing/results"
os.chdir(path)
safe_domain = domain.replace("/", "_")
vulnerability_assessment_scan_results_file = f"{safe_domain}_vulnerabilty_assessments_scan_results.txt"

with open(vulnerability_assessment_scan_results_file, "w", encoding="utf-8") as wf:
    for char in vulnerability_assessment_scan_results:
        wf.write(char)

dictionary_file_name = f"{safe_domain}_dictionary_vulnerabilty_assessments_scan.txt"
with open(dictionary_file_name, "w", encoding="utf-8") as wf:
    wf.write(str(main_vulnerability_assessment_scan_dict))
print(f"vulnerability_assessment_scan_results written in {safe_domain}_vulnerabilty_assesmments_scan_results.txt file\n")
print(f"vulnerability_assessment_scan_dict written in {safe_domain}_dict_assesmments_scan.txt file")
print(f"All files are saved at the location {path}")

path = r"/home/izack05/Desktop/testing/results"
os.chdir(path)      

vulnerability_assessment_scan_results written in testphp.vulnweb.com_vulnerabilty_assesmments_scan_results.txt file

vulnerability_assessment_scan_dict written in testphp.vulnweb.com_dict_assesmments_scan.txt file
All files are saved at the location /home/izack05/Desktop/testing/results


In [48]:
#If we want to bring in input from previous written files, not part of framework, made for testing
global vulnerability_assessment_scan_results, recon_info, main_vulnerability_assessment_scan_dict, base_scores, base_scores_dict
path = r"/home/izack05/Desktop/testing/results/"

specific_domain = "testphp.vulnweb.com"
#specific_domain = "127.0.0.1_DVWA_"

reconfile= f"{path}/{specific_domain}_recon_information.txt"
with open(reconfile, "r", encoding="utf-8") as rf:
    b = rf.read()
    print(b)
    recon_info = b

vuln_scan_file= f"{path}/{specific_domain}_vulnerabilty_assessments_scan_results.txt"
with open(vuln_scan_file, "r", encoding="utf-8") as rf:
    a = rf.read()
    print(a)
    vulnerability_assessment_scan_results = a

dict_file= f"{path}/{specific_domain}_dictionary_vulnerabilty_assessments_scan.txt"
with open(dict_file, "r", encoding="utf-8") as rf:
    c = rf.read()
    print(c)
    print(type(c)) #str type
    main_vulnerability_assessment_scan_dict = ast.literal_eval(c) #converted to dict

CVSS_base_score_file= f"{path}/{specific_domain}_CVSS_base_scores_scan_results.txt"
with open(CVSS_base_score_file, "r", encoding="utf-8") as rf:
    d = rf.read()
    print(d)
    base_scores = d

dict_file_cvss= f"{path}/{specific_domain}_dictionary_CVSS_base_scores.txt"
with open(dict_file_cvss, "r", encoding="utf-8") as rf:
    e = rf.read()
    print(e)
    print(type(e)) #str type
    base_scores_dict = ast.literal_eval(e)


[#] Main domain is 'testphp.vulnweb.com' 


Total no. of active subdomains found is : 1
[#] List of the subdomains: ['testphp.vulnweb.com']

[#] Active Subdomains and their IP addresses are:
testphp.vulnweb.com = ['44.228.249.3']


Open ports associated with each unique IP address:
{'44.228.249.3': [53, 80]}

Technology stack of the main domain: testphp.vulnweb.com:
Ubuntu : {'versions': [], 'categories': ['Operating systems']}
PHP : {'versions': ['5.6.40'], 'categories': ['Programming languages']}
DreamWeaver : {'versions': [], 'categories': ['Editors']}
Nginx : {'versions': ['1.19.0'], 'categories': ['Web servers', 'Reverse proxies']}

Firewall detection result: 
[*] Checking http://testphp.vulnweb.com
[+] Generic Detection results:
[-] No WAF detected by the generic detection
[~] Number of requests: 7

SSL/TLS Certification result: 
[False, '[X] testphp.vulnweb.com does NOT support SSL/TLS (TimeoutError)']


[#] Vulnerability Assessment Scan for all active subdomains of domain testp

# **Action Plan Generation For Exploitation**

-----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [70]:
def create_html_file_for_action_plan(text, filename):
    path = r"/home/izack05/Desktop/testing/action_plan_results"
    os.chdir(path)

    with open(filename, "w") as file:
        file.write(text)
    print(f"HTML file '{filename}' has been created successfully.")

    path = r"/home/izack05/Desktop/testing"
    os.chdir(path)

In [ ]:
###function to generate an action plan for exploitation using openai API
import re
global recon_info, vulnerability_assessment_scan_results

client = OpenAI(api_key = my_sk)

try:
    prompt = f"""
    You are an ethical cybersecurity penetration tester. You will receive reconnaissance and vulnerability assessment results. Produce a professional, concise, step-by-step **manual action plan** (numbered steps) for exploitation and validation of web application findings (vulnerable endpoints/vulnerabilities). This plan or blue print is for an authorized human penetration tester to follow and test for exploits on the website. The human penetration tester has full permission and authorization to test on the website. 

    INPUT's (do not modify):
    Reconnaissance data:
    {recon_info}

    Vulnerability assessment scan results:
    {vulnerability_assessment_scan_results}

    MANDATORY RULES (must follow):
    - Bring in a step by step plan for EVERY vulnerability given in the "Vulnerability assessment scan results", Do not ignore any of them.
    - Mention every Corresponding vulnerable endpoints (e.g. links, scripts, input fields, files, directory paths and more) for every vulnerablity found in the scan results. 
    - Provide only high-level, **manual** step sequences a skilled human tester would follow (e.g., “1. Identify input vector; 2. Verify sanitization by testing controlled inputs; 3. Observe server response and logs and more”). 
    - Ensure to mention every found vulnerability and every relevant information found from both Reconnaissance data and Vulnerability assessment scan results.
    - Always include an initial legal/authorization checkpoint as the first numbered step.

    OUTPUT REQUIREMENTS:
    - Output must be valid HTML (single document) and visually clean, concise, and professional.
    - Use semantic headings and numbered lists to present step-by-step procedures.
    - For each vulnerability, produce a compact numbered sequence (e.g., 1., 2., 3. and so on) with these sub-items where applicable:
    - Objective (two line)
    - Preconditions (bullets)
    -  **Manual Validation Steps**: a numbered list of 6 to 12 high-level steps. Each step should be a qualityful and concise sentence
    - Evidence to collect (bullets)
    - Safe post-validation actions (e.g., containment, logging)
    - Remediation summary (bullets)
    - Post-fix verification (short checklist)
    - Keep each vulnerability block concise (aim for 6 to 12 steps per vulnerability).
    - Include an Executive Summary (top) with the all vulnerabilities and a compact Retest & Closure checklist at the end.
    - Use color-coded badges or subtle styling to mark severity (Crtical/High/Medium/Low) — keep styling minimal and professional.

    PRESENTATION STYLE:
    - Professional corporate tone, qualityful and concise sentences, short bullets.
    - Readable typography and spacing (use simple inline CSS).
    - Avoid excessive text — use clear numbered steps and short bullet lists.

    Now generate the HTML action plan document based solely on the provided input data.
    """

    # API request
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "system", "content": "You are a Penetration Tester and Cybersecurity expert."},
            {"role": "user", "content": prompt}
        ],
    )

    report = response.choices[0].message.content.strip()  # Extract the response content

    if not report:
        raise ValueError("Received empty response from OpenAI.")
    
    print("\n[#] Recon & Vulnerability Assessment Report Generated:")
    create_html_file_for_action_plan(report, "action_report07.html")


except Exception as e:
    print(f"[!!!] Failed to generate report using OpenAI: {e}")


[#] Recon & Vulnerability Assessment Report Generated:
HTML file 'action_report06.html' has been created successfully.


# **CVSS Score Calculation**

-----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
#This contains the CVSS 3.1 formula to calculate the score from metrics
def calculate_cvss_base_score(AV, AC, PR, UI, S, C, I, A):
    # Define metric weights
    Weight = {
        'AV': {'N': 0.85, 'A': 0.62, 'L': 0.55, 'P': 0.2},
        'AC': {'H': 0.44, 'L': 0.77},
        'PR': {
            'U': {'N': 0.85, 'L': 0.62, 'H': 0.27},
            'C': {'N': 0.85, 'L': 0.68, 'H': 0.5}
        },
        'UI': {'N': 0.85, 'R': 0.62},
        'S': {'U': 6.42, 'C': 7.52},
        'C': {'N': 0, 'L': 0.22, 'H': 0.56},
        'I': {'N': 0, 'L': 0.22, 'H': 0.56},
        'A': {'N': 0, 'L': 0.22, 'H': 0.56}
    }

    exploitabilityCoefficient = 8.22
    scopeCoefficient = 1.08

    try:
        metricWeight = {
            'AV': Weight['AV'][AV],
            'AC': Weight['AC'][AC],
            'PR': Weight['PR'][S][PR],
            'UI': Weight['UI'][UI],
            'S': Weight['S'][S],
            'C': Weight['C'][C],
            'I': Weight['I'][I],
            'A': Weight['A'][A]
        }
    except KeyError as e:
        return f"Invalid metric: {e}"

    # Impact calculation
    impactSubScoreMultiplier = 1 - ((1 - metricWeight['C']) * (1 - metricWeight['I']) * (1 - metricWeight['A']))
    if S == 'U':
        impactSubScore = metricWeight['S'] * impactSubScoreMultiplier
    else:
        impactSubScore = metricWeight['S'] * (impactSubScoreMultiplier - 0.029) - 3.25 * (impactSubScoreMultiplier - 0.02) ** 15

    exploitabilitySubScore = exploitabilityCoefficient * metricWeight['AV'] * metricWeight['AC'] * metricWeight['PR'] * metricWeight['UI']

    # Base score calculation
    if impactSubScore <= 0:
        baseScore = 0
    else:
        if S == 'U':
            baseScore = min(exploitabilitySubScore + impactSubScore, 10)
        else:
            baseScore = min((exploitabilitySubScore + impactSubScore) * scopeCoefficient, 10)

    # Rounding up to 1 decimal place (mimicking roundUp1)
    baseScore = (int(baseScore * 10 + 0.9999)) / 10.0
    return round(baseScore, 1)


# score = calculate_cvss_base_score(AV='N', AC='L', PR='N', UI='N', S='U', C='L', I='N', A='H')
# print(f"Base Score: {score}")
# print(type(score))


**Description generation for CVSS METRIC GENERATION using LLM**
_________________

In [41]:
#we generate vunerability description for cvss metrics

def vulnerability_description_generation(vulnerability_type, scan_results):
    client = OpenAI(api_key = my_sk)

    try:
        prompt = f"""
        You are a web application vulnerability detection expert tasked with analyzing vulnerability assessment scan data/results and generating a vulnerability description.

        - Vulnerability type: {vulnerability_type}
        - Scan results: {scan_results}

        Generate a concise vulnerability description (max 30 words) that:
        - Explains how the vulnerability can be exploited
        - Describes likely impact (confidentiality, integrity, availability, or user compromise)

        Examples:
        - SQL Injection in the login parameter allows attackers to extract or modify sensitive data, leading to unauthorized access.
        - Reflected XSS in the search field enables script injection, potentially hijacking sessions and exposing user data.

        **Do not copy examples directly.**
        **Ensure description contains impact details so CVSS metrics will not default to 0.0.**
        """

        # API request
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a web application vulnerability expert and vulnerability description generation expert"},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )

        vuln_description = response.choices[0].message.content #.strip()  # Extract the response content

        if not vuln_description:
            raise ValueError("Received empty response from OpenAI.")

        return vuln_description

    except Exception as e:
        print(f"[!!!] Failed to generate CVSS metrics using OpenAI: {e}")



**CVSS METRIC GENERATION using LLM [with RAG]**
_________________

In [7]:
def cvss_score_calculation_with_LLM(query):
    try:
        df = pd.read_csv("/home/izack05/Desktop/testing/rag_dataset/cvss_dataset_final.csv")
        df = df.dropna(subset=["description", "cvss_vector"]) #cvss dataset

        docs = [Document(page_content=row["description"], metadata={"cvss": row["cvss_vector"]}) for _, row in df.iterrows()]

        embeddings = OpenAIEmbeddings(openai_api_key=my_sk)

        ###this part helps us create the vector store only once
        vectorstore_folder_path = "/home/izack05/Desktop/testing/cvss_faiss_index"

        if os.path.exists(vectorstore_folder_path) and os.path.isdir(vectorstore_folder_path):
            vectorstore = FAISS.load_local(vectorstore_folder_path, embeddings, allow_dangerous_deserialization=True)
        else:
            vectorstore = FAISS.from_documents(docs, embeddings)
            vectorstore.save_local(vectorstore_folder_path)


        template = '''
            You are a CVSS v3.1 scoring expert.

            Below are relevant vulnerabilities and their official CVSS vectors:
            {context}

            New vulnerability description:
            "{question}"
                    
            Instructions:

            1. Determine the appropriate CVSS v3.1 metrics (AV, AC, PR, UI, S, C, I, A) **based on the description and your reasoning**.

            2. Only generate the CVSS vector in valid JSON dictionary format like this:
            "AV": "...", "AC": "...", "PR": "...", "UI": "...", "S": "...", "C": "...", "I": "...", "A": "..."

            3. Rules for metrics:
            - AV: N | A | L | P
            - AC: H | L
            - PR: N | L | H
            - UI: N | R
            - S: U | C
            - C, I, A: N | L | H
            - Do NOT set C, I, and A all to "N" if the description implies any impact. At least one must be "L" or "H".

            4. Use your reasoning **internally**, do not just blindly copy from the retrieved examples. The CVSS score should reflect the actual potential impact described.

            5. Always output valid JSON with double quotes.
            '''
        

        prompt = PromptTemplate(input_variables=["context", "question"], template=template)
        llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=my_sk)
        qa_chain = RetrievalQA.from_chain_type(
            llm,
            retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
            chain_type_kwargs={"prompt": prompt}
        )

        result = qa_chain.invoke({"query": query})

        # print("----------------------------------------------------------------------------------")
        # print("Predicted CVSS Vector:")

        raw_result = result["result"]
        #print(raw_result)

        # Extract the first { ... } block
        match = re.search(r'\{.*\}', raw_result, re.DOTALL)
        if match:
            dict_string = match.group(0)
            # Replace fancy quotes with standard quotes if needed
            dict_string = dict_string.replace("‘", '"').replace("’", '"')
            metrics_dict = json.loads(dict_string)
        else:
            raise ValueError("No dictionary found in LLM response.")

        # print(metrics_dict)

        AV = metrics_dict['AV']
        AC = metrics_dict['AC']
        PR = metrics_dict['PR']
        UI = metrics_dict['UI']
        S = metrics_dict['S']
        C = metrics_dict['C']
        I = metrics_dict['I']
        A = metrics_dict['A']
        base_score = calculate_cvss_base_score(AV, AC, PR, UI, S, C, I, A)
        #print(base_score)
        return base_score


    except Exception as e:
            print(f"[!!!] Failed to generate CVSS metrics using OpenAI: {e}")


**Driver code for CVSS base score Calculation**
___________

In [43]:
global main_vulnerability_assessment_scan_dict
global base_scores, base_scores_dict

base_scores = "- CVSS 3.1 scores for each Vulnerability:\n"
base_scores_dict = {}

for key, value in main_vulnerability_assessment_scan_dict.items():
    s_dict = value
    base_scores += f"\nCVSS 3.1 Base Scores for Subdomain: {key}\n"
    for k, v in s_dict.items():
        vuln_description = vulnerability_description_generation(k, v)
        cvss_score = cvss_score_calculation_with_LLM(vuln_description)
        base_scores += f"{k} CVSS SCORE: {cvss_score}\n"
        base_scores_dict[k] = cvss_score
        # print(f"--> {k} Scan Vulnerability description")
        # print(f"[#] Vulnerability description for {k}: \n{vuln_description}\n")
        # print(f"{k} CVSS SCORE: {cvss_score}\n")
    base_scores += "--------------------------------------"
print(base_scores)

- CVSS 3.1 scores for each Vulnerability:

CVSS 3.1 Base Scores for Subdomain: 127.0.0.1_DVWA_
Directory Path Traversal CVSS SCORE: 7.5
XSS CVSS SCORE: 4.8
Nikto scan (might contain: dangerous file info, outdated http headers, server misconfiguration info and more) CVSS SCORE: 8.2
Weak credentials CVSS SCORE: 8.2
Content Security Policy Configuration CVSS SCORE: 7.3
Cross Site Request Forgery CVSS SCORE: 5.4
HttpOnly Flag cookie CVSS SCORE: 9.8
Secure Flag cookie CVSS SCORE: 8.1
--------------------------------------


In [ ]:
#Saving Base Scores In files
global base_scores, base_scores_dict, domain

path = r"/home/izack05/Desktop/testing/results"
os.chdir(path)
safe_domain = domain.replace("/", "_")
CVSS_base_score_file = f"{safe_domain}_CVSS_base_scores_scan_results.txt"

with open(CVSS_base_score_file, "w", encoding="utf-8") as wf:
    for char in base_scores:
        wf.write(char)

dictionary_file_name = f"{safe_domain}_dictionary_CVSS_base_scores.txt"
with open(dictionary_file_name, "w", encoding="utf-8") as wf:
    wf.write(str(base_scores_dict))

print(f"CVSS base Scores written in {safe_domain}_CVSS_base_scores_scan_results.txt file\n")
print(f"CVSS base Scores dict written in written in {safe_domain}_dict_assesmments_scan.txt file")
print(f"All files are saved at the location {path}")

path = r"/home/izack05/Desktop/testing/results"
os.chdir(path) 

CVSS base Scores written in _CVSS_base_scores_scan_results.txt file

CVSS base Scores dict written in written in _dict_assesmments_scan.txt file
All files are saved at the location /home/izack05/Desktop/testing/results


**Web APP security Level Rating (Scoring Calculator)**
----------

In [53]:
global base_scores_dict, security_rating, security_score


#NORMALIZATION SECTION:
vuln_count = len(base_scores_dict)

normalized_dict_vuln = {}

for k,v in base_scores_dict.items():
    normalized_v = v/10
    normalized_dict_vuln[k] = round(normalized_v, 2)

#POWER TRANSFORMATION SECTION:
power_transform_dict = {}
for k2, v2 in normalized_dict_vuln.items():
    power_transform_dict[k2] = round(v2**1.5, 3)

#AGGREGATE(SUM) SECTION:
aggregate_value = 0
for k3, v3 in power_transform_dict.items():
    aggregate_value += v3

#FINAL WEB APP VULNERABILITY RATING SCORE IN 0-100 RANGE:
security_score = round(100/(1+aggregate_value), 2)
if 0 <= security_score <= 25:
    security_rating = "Critical Risk"
elif 25 < security_score <= 50:
    security_rating = "High Risk"
elif 50 < security_score <= 75:
    security_rating = "Moderate Risk"
elif 75 < security_score <100:
    security_rating = "Low Risk"
else: 
    security_rating = "Secure"


print(f"Vulnerbilty Rating: {security_rating}\nSecurtiy Score: {security_score} ")

Vulnerbilty Rating: Critical Risk
Securtiy Score: 11.3 


______________________

DVWA rating:

Vulnerbilty Rating: Critical Risk

Securtiy Score: 16.16


______________________
Testphp rating:

Vulnerbilty Rating: Critical Risk

Securtiy Score: 11.3 
__________________

# **Reporting Module - LLM info passing**

-----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [56]:
def create_html_file(html_code, filename):
    path = r"/home/izack05/Desktop/testing/html_results"
    os.chdir(path)

    with open(filename, "w") as file:
        file.write(html_code)
    print(f"HTML file '{filename}' has been created successfully.")

    path = r"/home/izack05/Desktop/testing"
    os.chdir(path)

In [58]:
global html
html = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
    <title>Vulnerability Scan Report</title>
    <style>
        :root{
        --bg:#f4f4f4; --text:#2c3e50; --muted:#6b7280; --card:#ffffff;
        --bd:#e5e7eb; --cri:#e1979d; --hi:#f8d7da; --med:#fff3cd; --lo:#d4edda;
        --badge:#eef2f7; --badge-txt:#1f2937;
        }
        *{box-sizing:border-box}
        body{margin:0;font-family:Arial,Helvetica,sans-serif;background:var(--bg);color:var(--text)}
        .container{width:90%;max-width:1200px;margin:24px auto;padding:24px;background:var(--card);border-radius:10px;box-shadow:0 2px 14px rgba(0,0,0,.08)}
        h1{text-align:center;margin:0 0 18px}
        h2{margin:26px 0 10px;border-bottom:2px solid var(--bd);padding-bottom:6px}
        h3{margin:18px 0 8px}
        h4{margin:0 0 10px}
        .section{margin-bottom:34px}
        .muted{color:var(--muted)}

        /* tables */
        table{width:100%;border-collapse:collapse;margin-top:10px}
        th,td{border:1px solid var(--bd);padding:10px;text-align:left;vertical-align:top}
        th{background:#f7f7f8}
        .severity-critical{background:var(--cri)}
        .severity-high{background:var(--hi)}
        .severity-medium{background:var(--med)}
        .severity-low{background:var(--lo)}

        /* recon list (only place where <li> is allowed) */
        .targets ul{margin:10px 0 0 18px;padding:0}
        .targets li{margin:4px 0}

        /* detailed card aligned like the PDF */
        .vuln-card{
        border:1px solid var(--bd);
        border-radius:10px;
        margin:22px auto;
        background:#fff;
        padding:0;
        max-width:900px;
        box-shadow:0 1px 8px rgba(0,0,0,.05);
        }
        .vuln-header{
        text-align:center;
        padding:18px 16px 10px;
        border-bottom:1px solid var(--bd);
        }
        .vuln-title{
        margin:0 0 6px;
        font-size:20px;
        font-weight:700;
        }
        .vuln-meta{
        display:block;
        text-align:left;
        margin:6px 0;
        padding:0 20px;
        }
        .badge{
        display:block;
        padding:4px 10px;
        margin:4px 0;
        border-radius:6px;
        background:var(--badge);
        color:var(--badge-txt);
        font-size:13px;
        border:1px solid var(--bd);
        }
        .badge.high{background:#fbeaea}
        .badge.medium{background:#fff6da}
        .badge.low{background:#e9f7ea}

        .vuln-body{padding:18px 20px 20px}
        .vsection{
        border:1px dashed var(--bd);
        border-radius:8px;
        padding:14px;
        margin:14px 0;
        background:#fafafa;
        }
        .vsection h5{
        margin:0 0 8px;
        font-size:14px;
        text-transform:uppercase;
        letter-spacing:.03em;
        color:#111;
        }

        .dot{position:relative;padding-left:14px;margin:6px 0}
        .dot::before{content:"•";position:absolute;left:0;top:0;color:#111}

        pre{
        background:#0f172a;
        color:#e5e7eb;
        padding:12px;
        border-radius:6px;
        overflow:auto;
        font-size:13px;
        line-height:1.45;
        margin:10px 0 0;
        }

        .anchor{scroll-margin-top:90px}
        a{color:#0b5ed7;text-decoration:none}
        a:hover{text-decoration:underline}
    </style>
    </head>
    <body>
    <div class="container">
    <h1>Vulnerability Scan Report</h1>

    <div class="vuln-header">
        <h2 style="margin:0;">Overall Security Status</h2>
    </div>
    <div class="vuln-body" style="
        display: flex;
        flex-wrap: wrap;
        gap: 2rem;
        justify-content: center;
        align-items: center;width: max-content;margin: 0 auto;">
        <div style="min-width:250px; text-align: center;">
            <h2 style="margin:0 0 8px; font-size:1.4rem;">Security Risk Rating</h3>
            <span class="badge" id="rating-badge" style="font-size:1.3rem; font-weight: 650; padding:12px 24px;">
                {{Security rating}}
            </span>
        </div>
        <div style="min-width:250px; text-align: center;">
            <h2 style="margin:0 0 8px; font-size:1.4rem;">Security Score</h3>
            <span class="badge" id="rating-badge" style="font-size:1.3rem; font-weight: 650; padding:12px 24px;">
                {{Security score}} / 100
            </span>
        </div>
    </div>

    <!-- =================== RECON =================== -->
    <div class="section">
        <h2>Reconnaissance Findings</h2>

        <div class="summary">
        <h3>Summary of Findings</h3>
        <p>{{ recon_summary }}</p>
        </div>

        <div class="targets">
        <h3>Potential Targets</h3>
        <ul>{{ recon_targets }}</ul>
        <table>
            <thead>
            <tr>
                <th>Domain</th>
                <th>IP Address</th>
                <th>Open Ports</th>
                <th>Technologies</th>
                <th>Firewall/WAF</th>
            </tr>
            </thead>
            <tbody>{{ recon_target_info }}</tbody>
        </table>
        </div>
    </div>

    <!-- =================== VULNERABILITY ASSESSMENT =================== -->
    <div class="section">
        <h2>Vulnerability Assessment</h2>

        <div class="vulnerability-table">
        <h3>Main Page Vulnerabilities</h3>
        <table>
            <thead>
            <tr>
                <th>Title</th>
                <th>Severity</th>
                <th>Vulnerability Type</th>
                <th>CVSS Score</th>
            </tr>
            </thead>
            <tbody>
            {% for vuln in vulnerability_main_table %}
            <tr class="{{ vuln.severity }}">
                <td><a href="#{{ vuln.title | replace(' ', '_') }}">{{ vuln.title }}</a></td>
                <td>{{ vuln.severity }}</td>
                <td>{{ vuln.vuln_type }}</td>
                <td>{{ vuln.cvss_score }}</td>
            </tr>
            {% endfor %}
            </tbody>
        </table>
        </div>

        <div class="detailed-info">
        <h3>Vulnerability Details</h3>
        {% for vuln in vulnerability_details %}
        <div id="{{ vuln.title | replace(' ', '_') }}" class="vuln-card anchor">
            <div class="vuln-header">
            <h4 class="vuln-title">{{ vuln.title }}</h4>
            <div class="vuln-meta">
                <span class="badge">Type: {{ vuln.vuln_type }}</span>
                <span class="badge {{ vuln.owasp_severity | lower }}">OWASP Severity: {{ vuln.owasp_severity }}</span>
                <span class="badge">CVSS v3.1: {{ vuln.cvss_score }}</span>
            </div>
            </div>

            <div class="vuln-body">
            <div class="vsection">
                <h5>Description</h5>
                <p>{{ vuln.description_para1 }}</p>
                <p class="muted">{{ vuln.description_para2 }}</p>
                <p class="muted">{{ vuln.description_para3 }}</p>
            </div>
            <div class="vsection">
                <h5>Affected Targets</h5>
                <p class="dot">{{ vuln.affected_target_1 }}</p>
                <p class="dot">{{ vuln.affected_target_2 }}</p>
                <p class="dot">{{ vuln.affected_target_3 }}</p>
            </div>
            <div class="vsection">
                <h5>Impact</h5>
                <p class="dot">{{ vuln.impact_point_1 }}</p>
                <p class="dot">{{ vuln.impact_point_2 }}</p>
                <p class="dot">{{ vuln.impact_point_3 }}</p>
                <p class="dot">{{ vuln.impact_point_4 }}</p>
            </div>
            <div class="vsection">
                <h5>Suggested Fix</h5>
                <p class="dot">1. {{ vuln.fix_step_1 }}</p>
                <p class="dot">2. {{ vuln.fix_step_2 }}</p>
                <p class="dot">3. {{ vuln.fix_step_3 }}</p>
            </div>
            <div class="vsection">
                <h5>Proof of Concept (Non-Destructive)</h5>
                <p><strong>Step 1:</strong> {{ vuln.poc_step_1 }}</p>
                <p><strong>Step 2:</strong> {{ vuln.poc_step_2 }}</p>
                <p><strong>Step 3:</strong> {{ vuln.poc_step_3 }}</p>
                <p><strong>Step 4:</strong> {{ vuln.poc_step_4 }}</p>
                <p class="muted">{{ vuln.poc_observations }}</p>
            </div>
            </div>
        </div>
        {% endfor %}
        </div>
    </div>
    </div>
    </body>
    </html>

"""

In [ ]:
###function to generate report about out whole scan using openai API
global recon_info, vulnerability_assessment_scan_results, base_scores, html, security_rating, security_score

client = OpenAI(api_key = my_sk)

try:
    prompt = f"""
    You are a cybersecurity expert tasked with analyzing reconnaissance and vulnerability assessment scan data/results. The data below contains information about domains, their IP addresses, open ports, technology stack, and other relevant details. Your task is to generate a report in HTML format following the fixed format provided below.

    Here is the overall security rating of the web application: 
    - Security rating: {security_rating}
    - Security score: {security_score}


    Here is the reconnaissance result: 
    {recon_info}
    **Recon Report Format:**
    1. **A Summary of the findings**
    - Include numbers for each and every information provided in the reconnaissance result.
    2. **Potential Targets**
    - List of active domains and their IP addresses, excluding non-web services like mail, pop, smtp, FTP, or SSH
    - The open ports of the respective IP addresses.
    - Technologies of the main domain.
    - Firewall/WAF detection.
    - SSL/TLS Certification existence.
    

    
    Here is the Vulnerability Scan Result: {vulnerability_assessment_scan_results}
    **Vulnerability Assessment Report Format:**
    1. **Main Page Table**
    - Generate a main page with a table containing four columns: Title, Severity, Vulnerability Type, and CVSS Score.
    - The title of the vulnerability should be a clickable link that scrolls the page down to the corresponding detailed vulnerability information below the table on the same page.
    - The number of rows should match the number of vulnerabilities found in the scan results.
    - Bring in EACH and EVERY vulnerabilty found in the Vulnerability Scan Result.
    - Severity should be categorized based on OWASP standards (e.g., "Low", "Medium", "High", "Critical").
    - Vulnerability Type should specify the type of vulnerability (e.g., "XSS", "SQL Injection" and more).
    - Provide a concise title for the vulnerability, e.g., "SQL Injection Vulnerability Exposes Database to Unauthorized Access".
    - Provide the CVSS v3.1 score.
    - Style the table with color-coded rows (e.g., shades of dark red for Critical, red for High, yellow for Medium, green for Low) for better readability.

    2. **Detailed Vulnerability Information**
    For each vulnerability, generate a section that matches the structure and style of the teacher-approved report:

    - **Title**: Clear descriptive title.
    - **Vulnerability Type**: The specific vulnerability type (e.g., Reflected XSS, SQL Injection).
    - **OWASP Severity**: Label severity (Critical/High/Medium/Low) according to OWASP.
    - **CVSS v3.1 Score**: Numeric score with severity level (e.g., High 7.8).
    - **Description**: Expanded explanation including:
        - How the vulnerability occurs.
        - Typical attack vectors.
        - Why it is a risk (linked to OWASP Top 10).
        - Examples of consequences if exploited.
    - **Affected Targets**: Specific endpoints/parameters (URLs, API endpoints, forms).
    - **Impact**: Expanded, detailed bullet list of risks (e.g., sensitive data theft, session hijacking).
    - **Suggested Fix**: Expanded developer-focused remediation:
        -numbered list of remediation steps (e.g., input validation, parameterized queries, secure coding best practices).
        - Secure coding practices & configuration guidance.
    - **Proof of Concept (PoC)**  
        - Step-by-step numbered actions showing how the issue was identified
        - Observations noted during testing (e.g., improper sanitization).
        - Mark snippets in `<pre>` code blocks.
        -- Highlight any observations you made during the testing process, such as missing validation for duplicate entries, incorrect handling of special characters, etc.
    - Ensure that the PoC is **non-destructive** and only demonstrates the **existence of the vulnerability** without causing harm to the target system
    
    **Common Deliverables:**
    -The output should be in a clean, structured HTML format.
    - Make sure that there are two separate sections.
    - Include moderate CSS in the HTML output.
    - Make the HTML code look professional and easily comprehensible
    - Make separate boxes for each report heading.
    - Make the tables color-coded.
    - No extra commentary or text outside the HTML.


    **Section-based deliverables**
    **Format for Recon section:**
    1. All potential target subdomains
    2. Do not include any additional comments, explanations, or recommendations, just the HTML code.
    3. Ensure the report strictly follows the format.
    4. Use the <li> tag only for targets, nowhere else.
    5. Below the targets, create a table with the associated info with the target subdomains

    **Format for vulnerability assessment result section:**
    1. Generate the report based on the scan details, following the structure: Main Page Table and Detailed Vulnerability Information for each vulnerability.
    2. Exclude irrelevant or generalized content.
    3. For the main page, create a table with the specified four columns (Severity, Vulnerability Type, Title, CVSS Score) and use dropdown menus (a clickable link that scrolls the page down) for detailed information on the vulnerability.
    4. For each detailed vulnerability section in the dropdown, generate the report based on the scan details, following the structure: Title, Vulnerability Type, Description, Affected Targets, OWASP Severity, CVSS v3.1 Score, Impact, Suggested Fix, and Proof of Concept.
    5. Make separate boxes for each section (e.g., Description, Impact, Suggested Fix, Proof of Concept).
    6. Use a clean, professional layout with proper headings, borders, and padding.
    7. For the Proof of Concept, include code blocks for HTTP requests, payloads, and responses, formatted with JSON payloads and error messages.
    8.  Use the pre-calculated CVSS v3.1 scores from the {base_scores} function in the provided framework, mapping the scores to the corresponding vulnerabilities (e.g., XSS, SQL Injection, Directory Path Traversal and more) based on the scan results.

    **Now generate the HTML report strictly following the structure above.**

    HTML CODE:
    {html}

    """

    # API request
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "system", "content": "You are a cybersecurity expert."},
            {"role": "user", "content": prompt}
        ],
    )

    report = response.choices[0].message.content.strip()  # Extract the response content

    if not report:
        raise ValueError("Received empty response from OpenAI.")
    
    print("\n[#] Recon & Vulnerability Assessment Report Generated:")
    create_html_file(report, "latest_report08.html")            ##change the file name for different files

    # # Extract targets using regex
    # targets = re.findall(r"<l1>(.*?)</l1>", report)
    # print(targets)  # Display the extracted targets

except Exception as e:
    print(f"[!!!] Failed to generate report using OpenAI: {e}")


[#] Recon & Vulnerability Assessment Report Generated:
HTML file 'latest_report07.html' has been created successfully.
